In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
import multiprocessing
from joblib import Parallel, delayed
from get_obb_gbb_from_coco import _gen_seg_mask, segmentation_to_rotation_bbox

In [2]:
train_file  = '/datasets/dataset/coco2017/annotations/instances_train2017.json'
train_image = '/datasets/dataset/coco2017/train2017'
val_file   = '/datasets/dataset/coco2017/annotations/instances_val2017.json'
val_image  = '/datasets/dataset/coco2017/val2017'

resize = (512,512)

#voc_cats = [
#    'airplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'dining table',
#    'dog', 'horse', 'motorcycle', 'person', 'potted plant', 'sheep', 'sofa', 'train', 'tv'
#]
voc_cats = ['cat', 'dog']

In [3]:
def translateRotation(rotation, width, height):
    if (width > height):
        rotation = rotation + 90
    else:
        rotation = rotation + 180
    rotation = rotation-90
    return rotation

In [4]:
def write_annotation(ann_file, img_path):
    coco = COCO(ann_file)
    catIds = coco.getCatIds(catNms=voc_cats)
    name_id = dict(zip(catIds, voc_cats))
    
    with open(ann_file.replace('.json', '_obb_cat_dog.csv'), 'w') as writer:

        writer.write('file_name,x1,y1,x2,y2,label,angle\n')

        def _write(index):
            
            img_metadata = coco.loadImgs(index)[0]
            if not os.path.exists(os.path.join(img_path, img_metadata['file_name'])):
                return
            annotation   = coco.loadAnns(coco.getAnnIds(imgIds=index))
            h_img = img_metadata['height']
            w_img = img_metadata['width']
            seg_mask, labels = _gen_seg_mask(annotation, h_img, w_img)

            obb_all, final_labels = [],[]

            MinPixels = 20	 # ignores regions that are very small -- must check!
            for i in range(1, seg_mask.max() + 1):
                if np.sum(seg_mask == i) < MinPixels:
                    continue
                    
                if not (labels[i-1] in catIds):
                    continue
                    
                mask_inst  = seg_mask.copy()
                # generates binary image with current object
                mask_inst[mask_inst!=i] = 0

                # draw contours and get OBBs
                contours,hierarchy = cv2.findContours(cv2.resize(mask_inst, resize), 1, 2)
                box = np.int0(np.concatenate(contours, axis=0))

                rect1  = cv2.minAreaRect(box) # ((x,y), (w,h), angle) in opencv style
                points = np.int0(cv2.boxPoints(rect1)) # 2x4 matrix, representing corner points
                bbox = [rect1[0][0], rect1[0][1], rect1[1][0], rect1[1][1], rect1[2]]

                obb_all.append(bbox)
                final_labels.append(labels[i-1])
            
            # write OBBs and labels
            for g,l in zip(obb_all, final_labels):
                x,y,w,h,angle = bbox
                w = max(w, h); h = min(w, h)
                x1 = x-w/2;x2 = x+w/2
                y1 = y-h/2;y2 = y+h/2
                angle = translateRotation(angle, w_img, h_img)
                
                writer.write(f"{img_metadata['file_name']},{x1},{y1},{x2},{y2},{name_id[l]},{angle}\n")
        
        # run writing
        num_cores = multiprocessing.cpu_count()
        with Parallel(n_jobs=num_cores, prefer="threads") as parallel:
            parallel(delayed(_write)(index) for index in tqdm(coco.imgs))

In [5]:
write_annotation(train_file, train_image)

loading annotations into memory...
Done (t=13.07s)
creating index...


  0%|          | 24/118287 [00:00<12:42, 155.15it/s]

index created!


100%|██████████| 118287/118287 [14:07<00:00, 139.64it/s]


In [6]:
write_annotation(val_file, val_image)

loading annotations into memory...


  0%|          | 24/5000 [00:00<00:25, 197.79it/s]

Done (t=0.42s)
creating index...
index created!


100%|██████████| 5000/5000 [00:35<00:00, 139.37it/s]


In [7]:
voc_ids = dict(zip(voc_cats, range(1,len(voc_cats)+1)))

with open('/datasets/dataset/coco2017/annotations/classes_cat_dog.csv', 'w') as file:
    
    file.write('name,id\n')
    
    for name,id in voc_ids.items():
        file.write(f'{name},{id}\n')